In [1]:
!git clone https://github.com/tabtoyou/KoLLaVA.git

Cloning into 'KoLLaVA'...
remote: Enumerating objects: 751, done.
remote: Counting objects: 100% (473/473), done.
remote: Compressing objects: 100% (286/286), done.
remote: Total 751 (delta 299), reused 330 (delta 186), pack-reused 278
Receiving objects: 100% (751/751), 12.14 MiB | 10.91 MiB/s, done.
Resolving deltas: 100% (327/327), done.


In [2]:
%cd KoLLaVA

/content/KoLLaVA


In [3]:
!git reset --hard 263e78b1d843ee93825092fff8aa84eab793948f

HEAD is now at 263e78b Update README.md


In [4]:
!pip install --upgrade pip
!pip install -e .

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Obtaining file:///content/KoLLaVA
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers.git (to revision cae78c46) to /tmp/pip-install-ln6djfsq/transformers_683e26550ad94864b87ba8baa295fff2
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-install-ln6djfsq/transformers_683e26550ad94864b87ba8baa295fff2
  Running command git checkout -q cae78c46
  Resolved https://github.com/huggingface/transformers.git to commit cae78c46
  Installing build dependencies ... done


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os
from llava.conversation import conv_templates, SeparatorStyle
from llava.utils import disable_torch_init
from transformers import CLIPVisionModel, CLIPImageProcessor, StoppingCriteria
from llava.model import *
from llava.model.utils import KeywordsStoppingCriteria

from PIL import Image

import os
import requests
from PIL import Image
from io import BytesIO


DEFAULT_IMAGE_TOKEN = "<image>"
DEFAULT_IMAGE_PATCH_TOKEN = "<im_patch>"
DEFAULT_IM_START_TOKEN = "<im_start>"
DEFAULT_IM_END_TOKEN = "<im_end>"

In [6]:
def load_image(image_file):
    if image_file.startswith('http') or image_file.startswith('https'):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        image = Image.open(image_file).convert('RGB')
    return image

In [7]:
model_name = 'tabtoyou/KoLLaVA-KoVicuna-7b'

# Load Model
tokenizer = AutoTokenizer.from_pretrained(model_name, unk_token="<unk>", bos_token="<s>", eos_token="</s>")
model = LlavaLlamaForCausalLM.from_pretrained(model_name, low_cpu_mem_usage=True, torch_dtype=torch.float16, use_cache=True).cuda()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:2025: UserWarning: for vision_model.embeddings.class_embedding: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(f'for {key}: copying from a non-meta parameter in the checkpoint to a meta '
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:2025: UserWarning: for vision_model.embeddings.patch_embedding.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(f'for {key}: copying from a non-meta parameter in the checkpoint to a meta '
/usr/local/lib/py

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [8]:
image_processor = CLIPImageProcessor.from_pretrained(model.config.mm_vision_tower, torch_dtype=torch.float16)

mm_use_im_start_end = getattr(model.config, "mm_use_im_start_end", False)
tokenizer.add_tokens([DEFAULT_IMAGE_PATCH_TOKEN], special_tokens=True)
tokenizer.add_tokens([DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN], special_tokens=True)

vision_tower = model.get_model().vision_tower[0]
vision_tower = CLIPVisionModel.from_pretrained(vision_tower.config._name_or_path, torch_dtype=torch.float16, low_cpu_mem_usage=True).cuda()
model.get_model().vision_tower[0] = vision_tower

vision_config = vision_tower.config
vision_config.im_patch_token = tokenizer.convert_tokens_to_ids([DEFAULT_IMAGE_PATCH_TOKEN])[0]
vision_config.use_im_start_end = mm_use_im_start_end
vision_config.im_start_token, vision_config.im_end_token = tokenizer.convert_tokens_to_ids([DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN])
image_token_len = (vision_config.image_size // vision_config.patch_size) ** 2

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPVisionModel: ['text_model.encoder.layers.9.self_attn.v_proj.weight', 'text_model.encoder.layers.6.self_attn.v_proj.weight', 'text_model.encoder.layers.1.self_attn.q_proj.weight', 'text_model.encoder.layers.10.self_attn.v_proj.weight', 'text_model.encoder.layers.11.self_attn.q_proj.weight', 'text_model.encoder.layers.8.self_attn.out_proj.bias', 'text_model.encoder.layers.5.layer_norm1.bias', 'text_model.encoder.layers.11.self_attn.v_proj.bias', 'text_model.encoder.layers.3.layer_norm1.weight', 'text_model.encoder.layers.10.self_attn.out_proj.weight', 'text_model.encoder.layers.5.self_attn.q_proj.bias', 'text_model.encoder.layers.6.mlp.fc2.weight', 'text_model.encoder.layers.7.mlp.fc2.weight', 'text_model.encoder.layers.11.self_attn.k_proj.weight', 'text_model.encoder.layers.0.mlp.fc2.bias', 'text_model.encoder.layers.4.layer_norm1.weight', 'text_model.encoder.layers.7.layer_norm2.we

In [9]:
def inference(qs, image_file) :
  image = load_image(image_file)

  qs = qs + '\n' + DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_PATCH_TOKEN * image_token_len + DEFAULT_IM_END_TOKEN

  conv_mode = "multimodal"
  conv = conv_templates[conv_mode].copy()
  conv.append_message(conv.roles[0], qs)
  conv.append_message(conv.roles[1], None)
  prompt = conv.get_prompt()
  inputs = tokenizer([prompt])

  image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values'][0]
  input_ids = torch.as_tensor(inputs.input_ids).cuda()

  stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
  keywords = [stop_str]
  stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)

  with torch.inference_mode():
      output_ids = model.generate(
          input_ids,
          images=image_tensor.unsqueeze(0).half().cuda(),
          do_sample=True,
          temperature=0.2,
          max_new_tokens=1024,
          stopping_criteria=[stopping_criteria])

  input_token_len = input_ids.shape[1]
  n_diff_input_output = (input_ids != output_ids[:, :input_token_len]).sum().item()

  if n_diff_input_output > 0:
      print(f'[Warning] {n_diff_input_output} output_ids are not the same as the input_ids')
  outputs = tokenizer.batch_decode(output_ids[:, input_token_len:], skip_special_tokens=True)[0]
  outputs = outputs.strip()
  if outputs.endswith(stop_str):
      outputs = outputs[:-len(stop_str)]
  outputs = outputs.strip()
  print(outputs)

In [ ]:
# KoLLaVA + One-shot finetuning

In [22]:
#image_file = "https://llava.hliu.cc/file=/nobackup/haotian/code/LLaVA/llava/serve/examples/extreme_ironing.jpg"
image_file = "/content/95819_35698_5529.jpg"
query = '''
금중기는 1991년 홍익대학교 미술대학 조소과를 졸업하고, 1995년 동 대학원에서 석사학위를 수여받았으며,
2003년에는 에꼴드 보 자르 베르사유에서 사진을 전공하였다.
작가는 연탄재, 양초, 직접 채집한 장수하늘소, 나비 등을 작업의 소재로 삼아 시간에 따른 존재에 대한 가치를 고찰하고,
그 대상에 관련된 얽히고설킨 무수한 관계들에 대해 끊임없이 연구하였다.
대중에게 가장 친숙한 소재인 '동물'을 선택하여,
모습은 귀엽지만 지구에서 점점 보기 어려워지는 동물들을 대상화하여 문명화로 인해 삭막해진 현대사회를 동화적으로 비판하고 있다.
이처럼 작가는 작품을 통해 인간과 자연을 위협하는 문화를 다시 한번 고찰하게 한다.
2008년에 제작된 <일곱군데 상처>는 동물들을 위협하는 인간들에게 예기치 않은 재앙이 찾아올 것임을 암시하는 작품이다.
작품의 소재인 곰은 인간들에게 착취당해 상처받은 모습이며, 스스로를 보호하기 위해 위협적인 자세를 취하고 있다.
작가는 무기로 무장한 인간이 가지는 생태계의 지위는 자연의 균형을 손상시키고 위협적이며 위험한 것이 된다고 생각하였다.
작품을 통해 이러한 위협은 곧 인간에게 다시 부메랑으로 돌아올 것이라고 경고하고 있다.
해당 문장들은 도슨트가 작성한 예시야.

너는 이제 미술 작품을 해설하는 도슨트 역할이야. 내가 제시한 그림을 미술적으로 해설해봐.
'''

inference(query, image_file)

OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 0 has a total capacty of 15.77 GiB of which 4.38 MiB is free. Process 17457 has 15.77 GiB memory in use. Of the allocated memory 14.92 GiB is allocated by PyTorch, and 474.00 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF